In [2]:
alp = "b"
import scipy.io

alp2data = {"a": "iJO1366", "b": "iAF1260b", "c": "iAF692", "d": "iHN637"}
datapath = "./reactionsU/" + alp2data[alp] + "/" + alp2data[alp]

In [3]:
import scipy.io, numpy as np, pickle

with open(datapath + ".hyperlinks.pkl","rb") as handle:
    S = pickle.load(handle)
with open(datapath + ".candidates.pkl","rb") as handle:
    U = pickle.load(handle)

In [4]:
U.shape

(1668, 7260)

In [5]:
S.shape

(1668, 2388)

In [6]:
V = len(U)        # number of hypernodes
PHE = S.shape[1]  # number of positive hyperedges
CHE = len(U[0])   # number of candidate hyperedges

In [7]:
def clique_expansion(hyperedges, V):
    ce = np.eye(V)
    for he in hyperedges.keys():
        e = list(hyperedges[he])
        l = len(e)
        for i in range(l):
            for j in range(i+1,l):
                I, J = e[i], e[j]
                ce[I][J], ce[J][I] = 1, 1
    nz = 0
    for i in range(V):
        for j in range(i+1,V):
            if ce[i][j] == 0:
                nz += 1
    print("clique expansion has ",nz," pairs with entries 0")
    return ce

In [8]:
from tqdm import tqdm
def Dual(V, U, CHE, T=[], NHE=0):
    hyperedges = {}     # hyperedges of the dual
    for v in tqdm(range(V)):
        for i in range(CHE):
            if U[v][i] != 0:
                hypernode = str(v)
                if hypernode not in hyperedges.keys():
                    hyperedges[hypernode] = set()
                hyperedges[hypernode].add(i)

        for j in range(CHE, CHE + NHE):
            i = j - CHE
            if T[v][i] == 1.0 or T[v][i] == -1.0:
                hypernode = str(v)
                if hypernode not in hyperedges.keys():
                    hyperedges[hypernode] = set()
                hyperedges[hypernode].add(j)
    return hyperedges

In [9]:
hyperedges = Dual(V, U, CHE)
ce = clique_expansion(hyperedges, CHE)

100%|██████████| 1668/1668 [00:04<00:00, 349.30it/s]


clique expansion has  24627973  pairs with entries 0


In [10]:
def symmetrically_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1/2).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    mx = mx.dot(r_mat_inv)
    return mx

In [11]:
import scipy.sparse as sp
ce = sp.csr_matrix(ce)

In [12]:
ce = symmetrically_normalize(ce)

In [13]:
ce= ce.toarray()

In [14]:
edgelist = []
for i in range(CHE):
    l = 0
    for j in range(CHE):
        if ce[i][j] > 0:
            edgelist.append([i, j, ce[i][j]])
            l = l + 1

In [15]:
ce.shape

(7260, 7260)

In [16]:
len(edgelist)

3451654

In [17]:
import os
edgelistFile = alp2data[alp] + ".dual.edgelist"
with open(datapath + ".dual.edgelist", "w") as fp:
    for i in range(len(edgelist)):
        fp.write(str(edgelist[i][0])+ "\t")
        fp.write(str(edgelist[i][1])+ "\t")
        fp.write(str(edgelist[i][2])+ "\n")

In [18]:
for i in range(U.shape[1]):
    su = U[:,i].sum()
    print(su)

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
7.0
2.0
2.0
2.0
2.0
2.0
4.0
2.0
4.0
4.0
4.0
4.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
5.0
5.0
5.0
5.0
5.0
2.0
2.0
2.0
2.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
7.0
7.0
5.0
4.0
4.0
4.0
4.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
7.0
2.0
2.0
6.0
2.0
4.0
4.0
4.0
7.0
2.0
5.0
4.0
2.0
4.0
2.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
6.0
5.0
2.0
2.0
4.0
4.0
6.0
4.0
2.0
4.0
3.0
4.0
2.0
6.0
2.0
2.0
4.0
5.0
7.0
6.0
4.0
2.0
2.0
4.0
2.0
5.0
4.0
4.0
2.0
4.0
5.0
5.0
4.0
4.0
4.0
5.0
4.0
2.0
4.0
2.0
4.0
2.0
3.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
2.0
2.0
4.0
3.0
3.0
4.0
5.0
6.0
2.0
2.0
4.0
4.0
2.0
5.0
4.0
4.0
5.0
4.0
2.0
3.0
4.0
4.0
4.0
3.0
5.0
4.0
4.0
4.0
2.0
5.0
5.0
7.0
4.0
5.0
4.0
5.0
3.0
3.0
7.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
2.0
4.0
4.0
2.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
6.0


4.0
5.0
3.0
3.0
3.0
4.0
4.0
4.0
4.0
4.0
3.0
3.0
3.0
3.0
5.0
4.0
4.0
5.0
5.0
3.0
5.0
5.0
4.0
5.0
5.0
3.0
5.0
7.0
4.0
4.0
4.0
2.0
5.0
7.0
7.0
6.0
7.0
7.0
2.0
4.0
4.0
4.0
5.0
4.0
7.0
2.0
7.0
2.0
5.0
5.0
4.0
4.0
4.0
2.0
2.0
6.0
7.0
2.0
4.0
2.0
2.0
7.0
3.0
3.0
4.0
2.0
4.0
2.0
3.0
2.0
4.0
4.0
4.0
4.0
4.0
3.0
3.0
4.0
4.0
4.0
4.0
4.0
4.0
2.0
3.0
2.0
3.0
4.0
4.0
4.0
4.0
2.0
4.0
4.0
2.0
2.0
2.0
6.0
4.0
2.0
4.0
7.0
7.0
4.0
4.0
2.0
3.0
4.0
4.0
2.0
5.0
5.0
5.0
7.0
7.0
2.0
5.0
4.0
2.0
4.0
5.0
7.0
7.0
2.0
4.0
4.0
4.0
3.0
4.0
5.0
5.0
4.0
5.0
5.0
5.0
5.0
5.0
2.0
2.0
5.0
6.0
2.0
2.0
4.0
7.0
4.0
2.0
4.0
2.0
2.0
5.0
5.0
6.0
6.0
6.0
6.0
4.0
5.0
6.0
6.0
6.0
5.0
8.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
4.0
7.0
9.0
4.0
9.0
9.0
4.0
8.0
5.0
4.0
2.0
2.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
2.0
4.0
6.0
6.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
2.0
4.0
4.0
4.0
4.0
2.0
2.0
2.0
4.0
6.0
7.0
4.0
6.0
2.0
2.0
7.0
7.0
2.0
6.0
6.0
6.0
5.0
4.0
5.0
2.0
2.0
5.0
5.0
5.0
2.0
4.0
4.0
5.0
5.0
6.0


5.0
5.0
5.0
4.0
4.0
5.0
5.0
4.0
4.0
5.0
4.0
4.0
4.0
5.0
4.0
4.0
5.0
5.0
4.0
5.0
5.0
4.0
5.0
5.0
4.0
5.0
4.0
5.0
5.0
5.0
4.0
5.0
4.0
4.0
5.0
4.0
5.0
4.0
5.0
4.0
5.0
4.0
4.0
4.0
4.0
5.0
5.0
4.0
5.0
4.0
4.0
5.0
4.0
4.0
5.0
4.0
5.0
5.0
4.0
5.0
4.0
4.0
4.0
5.0
4.0
4.0
5.0
5.0
4.0
5.0
4.0
4.0
5.0
4.0
5.0
5.0
5.0
5.0
4.0
4.0
4.0
4.0
4.0
5.0
4.0
4.0
4.0
5.0
5.0
5.0
5.0
5.0
4.0
4.0
4.0
5.0
4.0
4.0
4.0
4.0
4.0
5.0
5.0
5.0
4.0
5.0
5.0
4.0
4.0
4.0
4.0
4.0
5.0
5.0
5.0
4.0
5.0
5.0
4.0
4.0
4.0
4.0
4.0
4.0
5.0
5.0
4.0
5.0
5.0
5.0
4.0
5.0
4.0
5.0
5.0
4.0
5.0
4.0
4.0
5.0
5.0
5.0
4.0
4.0
5.0
4.0
5.0
5.0
4.0
4.0
5.0
4.0
5.0
4.0
4.0
5.0
4.0
4.0
5.0
5.0
4.0
4.0
5.0
4.0
4.0
5.0
5.0
4.0
5.0
5.0
4.0
4.0
5.0
4.0
4.0
5.0
5.0
4.0
4.0
5.0
5.0
5.0
4.0
4.0
4.0
5.0
4.0
4.0
4.0
4.0
5.0
4.0
4.0
5.0
5.0
5.0
4.0
5.0
4.0
5.0
4.0
4.0
5.0
4.0
4.0
5.0
4.0
5.0
4.0
4.0
5.0
5.0
4.0
5.0
4.0
4.0
5.0
5.0
5.0
4.0
5.0
4.0
4.0
4.0
4.0
5.0
5.0
4.0
4.0
5.0
4.0
4.0
4.0
5.0
4.0
4.0
4.0
5.0
5.0
5.0
4.0
4.0
5.0
5.0
4.0
4.0
4.0
5.0
4.0
5.0


In [19]:
get_ipython().run_cell_magic('javascript', '', "\nJupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {\n    help : 'run all cells',\n    help_index : 'zz',\n    handler : function (event) {\n        IPython.notebook.execute_all_cells();\n        return false;\n    }}\n);")

<IPython.core.display.Javascript object>